Module Imports

In [10]:
import os
import re
import csv
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_files
from sklearn.metrics import accuracy_score,classification_report
from sklearn.linear_model import LogisticRegression
from string import digits


Data imports and parsing

In [31]:
data_dir = '/content/drive/MyDrive/10blogs' #a dir of 10 blogs in xml
data_xml=[]
data_txt={}

In [12]:
for filename in os.listdir(data_dir):
    f = os.path.join(data_dir,filename)
    if os.path.isfile(f):
          data_xml.append(f)

In [13]:
for i in data_xml:
  tree = ET.parse(i)  #parsing
  root = tree.getroot()   #parsing
  textArr = []
  counter = 0
  for child in root:    
    if child.tag=='post':
      if len(child.text) >500:
        if counter < 2:
          textArr.append(str(child.text[:500]))
          data_txt[i]=textArr 
          counter+=1
  

In [14]:
texts = list(data_txt.values())
author = list(data_txt.keys())

for i in range(len(texts)):
  texts[i] = str(texts[i])

Limiting number of articles and characters inside each article


In [15]:
for i in range(len(texts)):
  string1= texts[i][:len(texts[i])//2]
  string2= texts[i][len(texts[i])//2:]
  texts[i] = [string1[:1500], string2[:1500]]

In [16]:
fixed_author = []
for i in range(10):
  fixed_author.append(author[i].strip('/content/drive/MyDrive/10blogs/'))
  

In [17]:
for i in range(len(texts)):

  print(len(texts[i]))

2
2
2
2
2
2
2
2
2
2


Creating a data frame

In [18]:
df = pd.DataFrame({'texts':texts,
                   'author':fixed_author})
df=df.explode('texts')

In [19]:
texts[2]

['["\\n\\n     \\n      You should realise that everywhere in London is a complete mystery to me and because of the diversity of the place it\'s an adventure every time I head in to town.  But even so, it\'s quite strange stepping out of a tube station in the middle of London, walking about 200 metres up the road, and suddenly feeling like you could be in some small town or village in the middle of Somerset (or somewhere equally not-London).  But that\'s what I managed to do yesterday as I walked through Hig",',
 " '\\n\\n     \\n      For someone new to London (or just visiting as I have in the past) the London Underground is a saviour (despite being hot, crowded and late). Get a travelcard and you can just from one place to another without any problem at all.    But the problem with navigating London via the tube is that it gives a totally skewed impression of the Geography of the place. People get the tube when walking would have taken half the time, if only they knew where they wer

In [20]:
xfeature = df['texts']
ylabel = df['author']

In [21]:
# list of documents
docs = xfeature
# create the transform
vectorizer = TfidfVectorizer()
# tokenize and build vocab
X = vectorizer.fit_transform(docs)
print('vocabulary: ', vectorizer.vocabulary_)
# summarize encoded vector
print('shape: ', X.shape)
print('vectors: ', X.toarray())

vocabulary:  {'hey': 284, 'today': 658, 'woke': 731, 'up': 681, 'got': 262, 'all': 21, 'dressed': 175, 'and': 31, 'went': 712, 'to': 657, 'ballet': 55, 'it': 324, 'from': 244, 'wuz': 739, 'sooo': 582, 'hungry': 297, 'after': 12, 'that': 629, 'ate': 50, 'like': 355, 'feast': 213, 'at': 49, 'the': 630, 'chinese': 107, 'place': 468, 'ollies': 436, 'have': 275, 'such': 606, 'good': 260, 'food': 231, 'makes': 372, 'in': 308, 'darien': 142, 'suck': 607, 'anywho': 38, 'so': 576, 'alot': 23, 'there': 634, 'came': 97, 'home': 289, 'intotown': 317, 'with': 728, 'kendall': 333, 'had': 270, 'big': 71, 'slice': 571, 'of': 432, 'pizza': 467, 'snapple': 575, 'then': 633, 'we': 704, 'whole': 719, 'box': 85, 'scooby': 543, 'doo': 171, 'snacks': 574, 'baskin': 58, 'robins': 523, 'shakes': 555, 'this': 642, 'within': 729, 'hrs': 295, 'hehe': 279, 'town': 662, 'these': 635, 'people': 462, 'were': 713, 'weirdest': 710, 'fuckin': 245, 'mood': 395, 'right': 520, 'now': 425, 'its': 325, 'insane': 311, 'extrem

In [22]:
X_train,X_test,y_train,y_test = train_test_split(X, ylabel, train_size=0.10, random_state=42)
                                            

In [23]:
X_train.shape

(2, 748)

Tests

In [24]:
clf = MultinomialNB() #classification
clf.fit(X_train , y_train)

MultinomialNB()

In [25]:
y_pred = clf.predict(X_test)
print(accuracy_score(y_test,clf.predict(X_test)))
print(classification_report(y_test,y_pred))

0.05555555555555555
                                          precision    recall  f1-score   support

     2000257.female.14.indUnk.Scorpio.xm       0.00      0.00      0.00         2
29253.female.26.HumanResources.Cancer.xm       0.00      0.00      0.00         2
          37626.male.24.indUnk.Gemini.xm       0.00      0.00      0.00         1
       40964.female.23.RealEstate.Leo.xm       0.08      1.00      0.15         1
      43781.female.25.Education.Aries.xm       0.00      0.00      0.00         2
 48638.male.36.Technology.Sagittarius.xm       0.00      0.00      0.00         2
          566970.male.27.indUnk.Libra.xm       0.00      0.00      0.00         2
   61176.male.33.Technology.Capricorn.xm       0.00      0.00      0.00         2
         67459.male.34.Arts.Capricorn.xm       0.00      0.00      0.00         2
             854.male.25.indUnk.Virgo.xm       0.00      0.00      0.00         2

                                accuracy                           0.06     

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
logit = LogisticRegression() #logistic regression
logit.fit(X_train,y_train)


LogisticRegression()

In [27]:
accuracy_score(y_test,logit.predict(X_test))

0.05555555555555555

In [28]:
f26hrcancer = ["Billy Zane  and Jaoquin Phoenix are half-brothers.  Their father may or may not be  the  Lazarus (he could just be  a  Lazarus- type ), this mystical paternity has granted them the ability to raise the recently deceased from the dead.    Do they have a mission?  Yes, to find out the fate of their father and, if he still lives, kill him and set him free from the bonds of the Earth because for every year the Dad is alive, a single soul must take his place in the afterlife.  A soul that would have otherwise continued to live.  Oh, the angst!  Are the brothers raising the right people from the dead? Or are they making things worse?  Would they be better off doing nothing, or not bringing people back and just killing Dad?  How do they find all of this out?  Does a little man who lives in a cave and wears a white robe with an equally white beard tell them?  No!  Their mothers belonged to a cult - both died in childbirth, of course - and they were raised by a woman who amounts to the cult's den mother (because even cults need someone to clean the bathrooms).    Is there a conflict between the brothers? Of course!  Jaoquin's character is afraid of what will be revealed and would rather turn back than continue on and Billy's character drags him forward by sheer inertia.    But who will play Dad?  That is the question.  "]
vect1 = vectorizer.transform(f26hrcancer).toarray()

m25indvirgo = ["For someone new to London (or just visiting as I have in the past) the London Underground is a saviour (despite being hot, crowded and late). Get a travelcard and you can just from one place to another without any problem at all.    But the problem with navigating London via the tube is that it gives a totally skewed impression of the Geography of the place. People get the tube when walking would have taken half the time, if only they knew where they were going. For example, in the past I've gone from Tottenham Court Road to Covent Garden by getting the Northern line to Leicester Square and then changing on to the Picadilly line. Now, that probably took about 25 minutes, but if I'd just walkd I could have done it in 15. Ish.         A couple of weeks ago I bought an A-Z which has put a few things into perspective, but for the most part I'm still stuck on tube-geography. Yesterday I came across a  urlLink handy little web page  which gives a number of tube maps, including a  urlLink geographically correct map of the underground .  Nice to see, but doesn't really give you an impression of how you can get from place to another.    But also included on that page is a PDF of the  urlLink geographical map superimprosed onto a street map ! It shows landmarks, train stations and railway lines, tube stop and lines, all in the correct locations, so now you really can see just how quick it would have been to walk instead of fighting the commute! "]
vect2 = vectorizer.transform(m25indvirgo).toarray()

m36techsagi =["Microsoft, it's long been known, has the digital music market (refined and defined by Apple) in its sights. Looks like they're getting close enough to the initiation of their plans that they're willing to start talking trash: "]
vect3 = vectorizer.transform(m36techsagi).toarray()


In [29]:
clf.predict(vect1)

array(['40964.female.23.RealEstate.Leo.xm'], dtype='<U33')

In [30]:
logit.predict(vect1)

array(['40964.female.23.RealEstate.Leo.xm'], dtype=object)